In [16]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [17]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [18]:
snp_dir = "/data/gpfs/home/tfaske/d/rabbruh/SNPcall"

In [19]:
bwa_dir = "/data/gpfs/home/tfaske/d/rabbruh/SNPcall/bwa"

In [20]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa


In [21]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [22]:
len(bam_files), bam_files[0]

(400,
 '/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_100_sorted.bam')

In [23]:
samtools = "samtools"
bcftools = "bcftools"
java = "java"
perl = "perl"
#perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

In [24]:
assembly = "/data/gpfs/assoc/denovo/tfaske/rabbruh/assembly/reference.fasta"

#indexed done by dDocent 

In [25]:
!$samtools faidx {assembly}

# Calculates coverage from bam_files
also outputs a file called "bam_coverage.csv" in the bam folder

In [26]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa


In [27]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [28]:
len(bam_files), bam_files[0]

(400,
 '/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_100_sorted.bam')

In [30]:
bam_names = []
cov_list = []
for i in range(0,len(bam_files)):
    bam = bam_files[i]
    print(bam)
    b = bam.split('/')[9] #set this 
    #print (b)
    !$samtools depth -a $b > $'cov.txt'
    cov = pd.read_csv('cov.txt', sep="\t",header=None)
    coverage = sum(cov.iloc[:,2])/len(cov.index)
    name = b.split('.F')[0]
    #print(name)
    bam_names.append(name)
    cov_list.append(coverage)
    print(coverage,name)
cov_df = pd.DataFrame(bam=bam_names,coverage=cov_list)
cov_df.head()

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_100_sorted.bam
17.86849933000777 EN_TF_100_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_101_sorted.bam
26.880883451585888 EN_TF_101_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_102_sorted.bam
30.86553182025473 EN_TF_102_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_103_sorted.bam
28.913262747221037 EN_TF_103_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_104_sorted.bam
23.663809440863005 EN_TF_104_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_105_sorted.bam
17.754194556529033 EN_TF_105_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_106_sorted.bam
22.943282051831456 EN_TF_106_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_107_sorted.bam
16.585821478727567 EN_TF_107_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_108_sorted.bam
28.29650022656289 EN_TF_108_sorted

22.868444504504563 EN_TF_167_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_168_sorted.bam
23.157938420132314 EN_TF_168_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_169_sorted.bam
17.125491780825502 EN_TF_169_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_16_sorted.bam
16.430330617637438 EN_TF_16_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_170_sorted.bam
9.851494819828927 EN_TF_170_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_171_sorted.bam
26.224258773842962 EN_TF_171_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_172_sorted.bam
11.929207252168148 EN_TF_172_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_173_sorted.bam
25.722275778338048 EN_TF_173_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_174_sorted.bam
23.492841422560634 EN_TF_174_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_175_sorted

24.973750969115635 EN_TF_233_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_234_sorted.bam
11.615116132224843 EN_TF_234_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_235_sorted.bam
14.796215084570798 EN_TF_235_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_236_sorted.bam
15.251884748350866 EN_TF_236_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_237_sorted.bam
18.569643356355645 EN_TF_237_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_238_sorted.bam
17.438230097072662 EN_TF_238_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_239_sorted.bam
22.934094821870485 EN_TF_239_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_23_sorted.bam
25.214624551270628 EN_TF_23_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_240_sorted.bam
11.63036810078065 EN_TF_240_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_241_sorted

25.843025910497623 EN_TF_2_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_300_sorted.bam
15.475116987236506 EN_TF_300_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_301_sorted.bam
20.85599988896845 EN_TF_301_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_302_sorted.bam
16.165386470532514 EN_TF_302_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_303_sorted.bam
2.6921723689448265 EN_TF_303_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_304_sorted.bam
11.26570846411494 EN_TF_304_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_305_sorted.bam
19.945383580125284 EN_TF_305_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_306_sorted.bam
22.083386777541197 EN_TF_306_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_307_sorted.bam
22.646912051941026 EN_TF_307_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_308_sorted.

24.917783541486667 EN_TF_367_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_368_sorted.bam
4.908527032233209 EN_TF_368_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_369_sorted.bam
11.19782511859028 EN_TF_369_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_36_sorted.bam
26.046097839524915 EN_TF_36_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_370_sorted.bam
26.306556494658334 EN_TF_370_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_371_sorted.bam
15.041851929427013 EN_TF_371_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_372_sorted.bam
18.92749246421696 EN_TF_372_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_373_sorted.bam
20.222060540949794 EN_TF_373_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_374_sorted.bam
24.209720631542414 EN_TF_374_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_375_sorted.b

16.08266364975472 EN_TF_73_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_74_sorted.bam
14.867107430643905 EN_TF_74_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_75_sorted.bam
13.278035492471407 EN_TF_75_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_76_sorted.bam
19.889378537932355 EN_TF_76_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_77_sorted.bam
8.187934273132727 EN_TF_77_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_78_sorted.bam
13.179525732451241 EN_TF_78_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_79_sorted.bam
5.370658401990073 EN_TF_79_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_7_sorted.bam
9.633403982557084 EN_TF_7_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_80_sorted.bam
8.887666327350855 EN_TF_80_sorted.bam
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_81_sorted.bam
8.483863917378935

EmptyDataError: No columns to parse from file

In [37]:
cov_dict = {"bam":bam_names,'coverage':cov_list}
cov_df = pd.DataFrame(cov_dict)
cov_df.head()
#len(cov_df)

,bam,coverage
0,EN_TF_100_sorted.bam,17.868499
1,EN_TF_101_sorted.bam,26.880883
2,EN_TF_102_sorted.bam,30.865532
3,EN_TF_103_sorted.bam,28.913263
4,EN_TF_104_sorted.bam,23.663809


In [38]:
cov_out = os.path.join(bwa_dir,'bam_coverage.csv')
cov_df.to_csv(path_or_buf=cov_out)

In [39]:
cov_df.coverage.describe()


count    399.000000
mean      17.309698
std        7.769467
min        0.891040
25%       11.694113
50%       17.433782
75%       23.019330
max       36.243055
Name: coverage, dtype: float64

In [40]:
len(cov_df.coverage)

399

# move good bams
    EN_TF_99 empty so only 399

In [56]:
len(cov_df[cov_df.coverage > 5]),len(cov_df[cov_df.coverage >= 3])

(370, 385)

In [57]:
good_bam = cov_df.bam[cov_df.coverage >= 3]
good_bam = good_bam.tolist()
len(good_bam),good_bam[0]

(385, 'EN_TF_100_sorted.bam')

In [58]:
good_bam_files = []
for i in range(0,len(good_bam)):
    bam = bwa_dir + '/' + good_bam[i]
    good_bam_files.append(bam)
len(good_bam_files), good_bam_files[1]

(385, '/data/gpfs/home/tfaske/d/rabbruh/SNPcall/bwa/EN_TF_101_sorted.bam')

In [47]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall


In [48]:
!mkdir 'good_bams'

In [49]:
good_bam_dir = os.path.join(snp_dir,'good_bams')
assert good_bam_dir

In [50]:
for i in range(0,len(good_bam_files)):
    good_bam = str(good_bam_files[i])
    !cp $good_bam $good_bam_dir

In [51]:
cd $good_bam_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/good_bams


In [52]:
good_bam_cp_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    good_bam_cp_files.append(x)
good_bam_cp_files = sorted(good_bam_files)

In [53]:
len(good_bam_cp_files),good_bam_cp_files[0]

(384,
 '/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/good_bams/EN_TF_100_sorted.bam')

# SNP CALL

In [60]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall


In [61]:
pwd

'/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall'

In [62]:
### select options
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'rabbruh'
time = '4-00:00:00' #time limit 4 days
cpus = 30
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [64]:
def write_callsnps_sh(account,partition,jobname,time,cpus,mem_cpu,email,assembly,good_bam_dir,snp_dir):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    with open("./run_callsnps.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_callsnps
#SBATCH --output output_callsnps.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s \n\n""" % (account,partition,time,int(cpus),int(mem_cpu),jobname,email))

        o.write("samtools mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR -Iugf %s %s/*sorted.bam | bcftools call -f GP,GQ -vmO z -o %s/samtools1-9.%s.vcf.gz" %
            (assembly,good_bam_dir,snp_dir,jobname))

In [65]:
write_callsnps_sh(account,partition,jobname,time,cpus,mem_cpu,email,assembly,good_bam_dir,snp_dir)

# run_call_snps.sh 
    cd /data/gpfs/home/tfaske/d/rabbruh/SNPcall
    source activate py36
    sbatch run_callsnps